# Introduction


Hello, my name is Jason Leff, and this jupyter notebook serves as a peek into how I created the dataset for my (ongoing) Thesis on Projecting NBA Three Point Percentage From Pre-NBA Data. The below groupings of cells will take you step-by-step through how I collected my data for this project, which was later exported into R to complete the analyses of my project.

# Relevant Links

These are links to various articles that I have learned from or I am using api from:
* https://github.com/swar/nba_api/tree/master
* https://medium.com/analytics-vidhya/intro-to-scraping-basketball-reference-data-8adcaa79664a
* https://stackoverflow.com/questions/55369159/how-to-perform-three-variable-correlation-with-python-pandas
* https://www.geeksforgeeks.org/exploring-correlation-in-python/
* https://stackoverflow.com/questions/29623171/simple-prediction-using-linear-regression-with-python
https://github.com/lbenz730/ncaahoopR
https://www.rdocumentation.org/packages/nbastatR/versions/0.1.110202031


# Exporting data from nba_api to csv

In [ ]:
!pip install nba_api #requires you to restart session so always do this first

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.3/285.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
# import modules
import pandas as pd
import datetime
import numpy as np
import math
import time
pd.__version__
from google.colab import files

# import packages from nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import CommonTeamRoster, CommonPlayerInfo, playercareerstats

teams_data = pd.DataFrame(teams.get_teams()) #get list of teams
player_info = pd.DataFrame() #empty df for player info
nba_data = pd.DataFrame() #empty df for nba data
change_list = ['FG','FG3','FT'] #list to calculate percentage difference

for id in teams_data['id']: #loop thru list of teams
  roster = CommonTeamRoster(id) #open page
  player_info = pd.concat([player_info, roster.get_data_frames()[0]]) #concat roster data to information dataframe
  for id in roster.get_data_frames()[0]['PLAYER_ID']: #loop through player ids from the roster data
    player = playercareerstats.PlayerCareerStats(id) #open the player page
    nba_data = pd.concat([nba_data, player.get_data_frames()[1]]) #concat player career data to information dataframe
    df = player.get_data_frames()[0] #pull database of every season player has played

    # Get player name for the print statement
    player_name = players.find_player_by_id(id)['full_name']
    print(f"Successfully processed data for player: {player_name}")

    for item in change_list: #iterate thru change_list
      column_to_analyze = item + '_PCT' #check pct column
      filter_column = item + 'A' #filter minimum attempts by using the 'A' column
      minimum_limit = 25 #a good cutoff point for minimum amount of attempts, I think
      filtered_df = df[df[filter_column] >= minimum_limit] #filter out rows with less than the minimum_limit attempts
      if len(filtered_df) >= 2: #check if there are two rows to subtract
        difference = filtered_df[column_to_analyze].max() - filtered_df[column_to_analyze].min() #subtract lowest value from highest value to (hopefully) determine growth
      else:
        difference = 0 #if not 2+ rows, set as NaN
      nba_data.loc[nba_data['PLAYER_ID'] == id,item + '_pct_inc'] = round(difference,3) #place the data in the right column and row of player_data
    time.sleep(60) #wait to not overload server (and not get blacklisted) # Reduced sleep time for demonstration

nba_data = nba_data.round(3)
nba_data.to_csv('nba_data.csv', index=False) #export data to csv
player_info.to_csv('player_info.csv', index=False)

files.download('nba_data.csv')
files.download('player_info.csv')

In [ ]:
# Round numeric columns to 3 decimal places before exporting
nba_data = nba_data.round(3)

nba_data.to_csv('nba_data.csv', index=False) #export data to csv
player_info.to_csv('player_info.csv', index=False)

In [ ]:
from google.colab import files
files.download('nba_data.csv')
files.download('player_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# import modules
import pandas as pd
import datetime
import numpy as np
import math
import time
pd.__version__
from google.colab import files, drive
drive.mount('/content/drive')

# import packages from nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import CommonTeamRoster, CommonPlayerInfo, playercareerstats

nba_data = pd.DataFrame() #empty df for nba data
#player_info = pd.DataFrame() #empty df for player info
player_info = pd.read_csv('https://docs.google.com/spreadsheets/d/1-TTl1BjdbS0YCUOVhiFwiNYCBjGcFXZ8HJsnFqxSEfI/gviz/tq?tqx=out:csv')
change_list = ['FG','FG3','FT'] #list to calculate percentage difference
seasons = list(range(2004, 2003, -1)) #this changes based on what data I am scraping for -- process was broken into many portions
teams_data = pd.DataFrame(teams.get_teams())


for season in seasons:
  # Initialize player_info for the current season
  player_info_season = pd.DataFrame()
  nba_data = pd.DataFrame() #empty df for nba data
  player_count = 0

  for id in teams_data['id']: #loop thru list of teams
    time.sleep(60)
    roster = CommonTeamRoster(team_id=id,season=season, league_id_nullable='00') #open page
    player_info_season = pd.concat([player_info_season, roster.get_data_frames()[0]]) #concat roster data to information dataframe


  #remove duplicates from the season's player_info
  #first remove blank EXP rows
  player_info_season = player_info_season[
    player_info_season['EXP'].notna() &
    (player_info_season['EXP'] != '') &
    (player_info_season['EXP'] != 'R')
  ]

  #then filter to only players not in player_info
  player_info_season = player_info_season[
    ~player_info_season['PLAYER_ID'].isin(player_info['PLAYER_ID'])
  ]

  player_info = pd.concat([player_info, player_info_season]).drop_duplicates(subset=['PLAYER_ID'], keep='first')

  last_player_id = player_info_season['PLAYER_ID'].iloc[-1]


  for id in player_info_season['PLAYER_ID']: #loop through player ids from the roster data
    player = playercareerstats.PlayerCareerStats(id) #open the player page
    new_data = player.get_data_frames()[1] #concat player career data to information dataframe
    df = player.get_data_frames()[0] #pull database of every season player has played

    #get player name for the print statement
    player_name = players.find_player_by_id(id)['full_name']
    print(f"Successfully processed data for player: {player_name}")

    for item in change_list: #iterate thru change_list
      column_to_analyze = item + '_PCT' #check pct column
      filter_column = item + 'A' #filter minimum attempts by using the 'A' column
      makes_column = item + 'M' #column for number of makes
      minimum_limit = 20 #a good cutoff point for minimum amount of attempts

      filtered_df = df[df[filter_column] >= minimum_limit] #filter out rows with less than the minimum_limit attempts

      if len(filtered_df) >= 2: #check if there are two rows to compare
        #initial percentage from first row in filtered dataframe
        initial_value = filtered_df[column_to_analyze].iloc[0]

        #calculate final value as total makes divided by total attempts across all filtered rows
        total_makes = filtered_df[makes_column].sum()
        total_attempts = filtered_df[filter_column].sum()
        final_value = total_makes / total_attempts if total_attempts > 0 else 0

        difference = final_value - initial_value
      else:
        difference = 0 #if not 2+ rows, set as 0

      new_data[item + '_pct_inc'] = round(difference, 3)

    nba_data = pd.concat([nba_data, new_data])

    time.sleep(60) #wait to not overload server

    player_count += 1
    if player_count % 50 == 0 or id == last_player_id: #save and download every 50 players or last in a season
      print(f"Saving and downloading data for season {season} after processing {player_count} players.")
      season_nba_data = nba_data[nba_data['SEASON_ID'].str.contains(str(season), na=False)].round(3) #filter nba_data for the current season
      season_player_info = player_info[player_info['PLAYER_ID'].isin(player_info_season['PLAYER_ID'])] #filter player_info for new players added this season
      nba_data.to_csv(f'nba_data_{season}_partial_{player_count}.csv', index=False) #export data to csv with season and count in filename
      player_info_season.to_csv(f'player_info_{season}_partial_{player_count}.csv', index=False) #export data to csv with season and count in filename

      file_list = [f'nba_data_{season}_partial_{player_count}.csv', f'player_info_{season}_partial_{player_count}.csv']

      for file in file_list:
        !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/Partials/"
        print(f"Saved {file} to Drive")
      time.sleep(10)

Mounted at /content/drive


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

## experimental code to fix the above block


In [ ]:
listy = ['203095','202722','1641735', '1641778', '1629716', '1630197']
for id in listy: #loop through player ids from the roster data
    player = playercareerstats.PlayerCareerStats(id) #open the player page
    nba_data = pd.concat([nba_data, player.get_data_frames()[1]]) #concat player career data to information dataframe
    df = player.get_data_frames()[0] #pull database of every season player has played

    # Get player name for the print statement
    player_name = players.find_player_by_id(id)['full_name']
    print(f"Successfully processed data for player: {player_name}")

    for item in change_list: #iterate thru change_list
      column_to_analyze = item + '_PCT' #check pct column
      filter_column = item + 'A' #filter minimum attempts by using the 'A' column
      makes_column = item + 'M' #column for number of makes
      minimum_limit = 25 #a good cutoff point for minimum amount of attempts

      filtered_df = df[df[filter_column] >= minimum_limit] #filter out rows with less than the minimum_limit attempts

      if len(filtered_df) >= 2: #check if there are two rows to compare
        initial_value = filtered_df[column_to_analyze].iloc[0]
       total_makes = df[makes_column].sum()
        total_attempts = df[filter_column].sum()
        final_value = total_makes / total_attempts if total_attempts > 0 else 0

        difference = final_value - initial_value
      else:
        difference = 0 #if not 2+ rows, set as 0

      nba_data.loc[nba_data['PLAYER_ID'] == id, item + '_pct_inc'] = round(difference, 3)

      time.sleep(60) #wait to not overload server

    nba_data.to_csv(f'nba_data_{season}_partial_{player_count}.csv', index=False) #export data to csv with season and count in filename
    player_info_season.to_csv(f'player_info_{season}_partial_{player_count}.csv', index=False) #export data to csv with season and count in filename

    file_list = [f'nba_data_{season}_partial_{player_count}.csv', f'player_info_{season}_partial_{player_count}.csv']

    for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/Partials/"
      print(f"Saved {file} to Drive")


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

## resume data collection


In [ ]:
import pandas as pd
import datetime
import numpy as np
import math
import time
pd.__version__
from google.colab import files

# import packages from nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import CommonTeamRoster, CommonPlayerInfo, playercareerstats

player_info = pd.read_csv('https://docs.google.com/spreadsheets/d/1-TTl1BjdbS0YCUOVhiFwiNYCBjGcFXZ8HJsnFqxSEfI/gviz/tq?tqx=out:csv')
teams_data = pd.DataFrame(teams.get_teams()) #get list of teams

for id in teams_data['id']: #loop thru list of teams
  roster = CommonTeamRoster(team_id=1610612741,season=2023,timeout=60) #open page
  player_info_season = pd.concat([player_info_season, roster.get_data_frames()[0]])
  break

  #remove duplicates from the season's player_info
player_info_season = player_info_season[~player_info_season['PLAYER_ID'].isin(player_info['PLAYER_ID'])]
  #append player info to the main player_info DataFrame
player_info = pd.concat([player_info, player_info_season], ignore_index=True)
player_info.head()

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612737,2024,0,Jacob Toppin,Jacob,jacob-toppin,0.0,F,6-8,200.0,"May 08, 2000",25.0,1.0,Kentucky,1631210,Signed on 03/04/25
1,1610612737,2024,0,Jalen Johnson,Jalen,jalen-johnson,1.0,F,6-8,219.0,"Dec 18, 2001",23.0,3.0,Duke,1630552,#20 Pick in 2021 Draft
2,1610612737,2024,0,Keaton Wallace,Keaton,keaton-wallace,2.0,G,6-3,185.0,"Feb 26, 1999",26.0,NaN,Texas-San Antonio,1630811,Signed on 09/29/23
3,1610612737,2024,0,Caris LeVert,Caris,caris-levert,3.0,G,6-6,205.0,"Aug 25, 1994",30.0,8.0,Michigan,1627747,NaN
4,1610612737,2024,0,Kobe Bufkin,Kobe,kobe-bufkin,4.0,G,6-4,195.0,"Sep 21, 2003",21.0,1.0,Michigan,1641723,#15 Pick in 2023 Draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,1610612740,2023,00,Cody Zeller,Cody,cody-zeller,40,F-C,6-11,240,"OCT 05, 1992",31.0,10,Indiana,203469,None
542,1610612741,2023,00,Andrew Funk,Andrew,andrew-funk,None,G,6-5,200,"SEP 21, 1999",24.0,R,Penn State,1641847,None
543,1610612741,2023,00,Onuralp Bitim,Onuralp,onuralp-bitim,17,F,6-6,215,"MAR 31, 1999",25.0,R,Bursaspor,1641931,None
544,1610612741,2023,00,Adama Sanogo,Adama,adama-sanogo,21,F,6-9,245,"FEB 12, 2002",22.0,R,UCONN,1641766,None


# Clean NBA Data


In [ ]:
import numpy as np
import pandas as pd
player_info = pd.read_csv('https://docs.google.com/spreadsheets/d/1-TTl1BjdbS0YCUOVhiFwiNYCBjGcFXZ8HJsnFqxSEfI/gviz/tq?tqx=out:csv')
nba_data = pd.read_csv('https://docs.google.com/spreadsheets/d/1KZUS5oSUvaUHmQZfMZ20RT3bOgOo6WQDBge6Mf8R0hI/gviz/tq?tqx=out:csv')

In [ ]:
player_info_cleaned = player_info.drop_duplicates(subset=['PLAYER_ID'], keep='first')
player_info_cleaned = player_info_cleaned.dropna(subset=['EXP']).reset_index(drop=True)
player_info_cleaned = player_info_cleaned.drop(columns=['index','TeamID','LeagueID','NICKNAME','HOW_ACQUIRED'], errors='ignore')

def convert_height_to_inches(row):
    height = row['HEIGHT']
    position = row['POSITION']

    if isinstance(height, str) and '-' in height:
        feet, inches = height.split('-')
        try:
            return int(feet) * 12 + int(inches)
        except ValueError:
            pass
    #heights ending in 0 did not load in the data, so we (reasonbly) guess what the height is in this function based on listed position
    if position in ['G', 'G-F']:
        return 72
    else:
        return 84

player_info_cleaned['HEIGHT_INCHES'] = player_info_cleaned.apply(convert_height_to_inches, axis=1)

In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

player_info_cleaned.to_csv('player_info_cleaned.csv')

file_list = ['player_info_cleaned.csv']
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved player_info_cleaned.csv to Drive


# Scrape sports-reference.com to csv


This block sorts players into college and not college players, then can be changed to sort the not college players into international and unavailable data players.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import pandas as pd
from google.colab import files, drive
drive.mount('/content/drive')

player_info = pd.read_csv('https://docs.google.com/spreadsheets/d/1-TTl1BjdbS0YCUOVhiFwiNYCBjGcFXZ8HJsnFqxSEfI/gviz/tq?tqx=out:csv')
players = player_info_cleaned[['PLAYER_SLUG', 'SCHOOL']] #setting the parameter to run the scraping function

successful_players = []
failed_players = []

for index, player in players.iterrows():
    player_slug = player['PLAYER_SLUG']
    school = player['SCHOOL']
    max_attempts = 13
    success = False
    is_ascii_error = False

    for attempt in range(1, max_attempts + 1):
        time.sleep(6) #server overloads if this is lower
        try:
            url = f"https://www.sports-reference.com/cbb/players/{player_slug}-{attempt}.html"
            html = urlopen(url)
            soup = BeautifulSoup(html, 'lxml')

            #check if the school name is in page to confirm the player
            if isinstance(school, str) and school.lower() in soup.get_text().lower():
                print(f"✅ Success: {player_slug} (Attempt {attempt}) - School matched: {school}")
                successful_players.append(player_slug + '-' + str(attempt))
                success = True
                break
            else:
                print(f"⚠️ Retrying {player_slug} (Attempt {attempt}): School not found in page.")

        except UnicodeEncodeError:
            print(f"❌ Failed: {player_slug} (Attempt {attempt}) - UnicodeEncodeError (ASCII issue)")
            is_ascii_error = True
            break
        except HTTPError as e:
            print(f"⚠️ Retrying {player_slug} (Attempt {attempt}): URL error ({e.code})")
        except Exception as e:
            print(f"⚠️ Retrying {player_slug} (Attempt {attempt}): An unexpected error occurred: {e}")

#check for ascii_errors
    if not success and not is_ascii_error:
        failed_players.append(player_slug)
        print(f"❌ Failed: {player_slug} after {max_attempts} attempts.")
    elif is_ascii_error:
        failed_players.append(player_slug)

failed_players1 = pd.DataFrame(failed_players, columns=['PLAYER_SLUG'])
successful_players1 = pd.DataFrame(successful_players, columns=['PLAYER_SLUG'])

failed_players1.to_csv("non-college_players.csv")
successful_players1.to_csv("college_players.csv")

file_list = ["non-college_players.csv", "college_players.csv"]
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Success: jacob-toppin (Attempt 1) - School matched: Kentucky
✅ Success: jalen-johnson (Attempt 1) - School matched: Duke
✅ Success: caris-levert (Attempt 1) - School matched: Michigan
✅ Success: kobe-bufkin (Attempt 1) - School matched: Michigan
⚠️ Retrying dyson-daniels (Attempt 1): URL error (404)
⚠️ Retrying dyson-daniels (Attempt 2): URL error (404)


KeyboardInterrupt: 

## experiment code

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import pandas as pd

player_info = pd.read_csv('https://docs.google.com/spreadsheets/d/1-TTl1BjdbS0YCUOVhiFwiNYCBjGcFXZ8HJsnFqxSEfI/gviz/tq?tqx=out:csv')
players = player_info[['PLAYER_SLUG', 'SCHOOL']] #setting the parameter to run the scraping function

failed_players = [] # Initialize failed_players as a list

for index, player in players.iterrows():
    player_slug = player['PLAYER_SLUG']
    school = player['SCHOOL']
    max_attempts = 5
    success = False
    is_ascii_error = False # Flag to check for ASCII error

    for attempt in range(1, max_attempts + 1):
        time.sleep(6) # Increased sleep time
        try:
            url = f"https://www.sports-reference.com/cbb/players/{player_slug}-{attempt}.html"
            html = urlopen(url)
            soup = BeautifulSoup(html, 'lxml')

            # Check if the school name exists in the page
            if isinstance(school, str) and school.lower() in soup.get_text().lower():
                print(f"✅ Success: {player_slug} (Attempt {attempt}) - School matched: {school}")
                success = True
                break  # Exit the attempt loop if successful
            else:
                print(f"⚠️ Retrying {player_slug} (Attempt {attempt}): School not found in page.")

        except UnicodeEncodeError:
            print(f"❌ Failed: {player_slug} (Attempt {attempt}) - UnicodeEncodeError (ASCII issue)")
            is_ascii_error = True
            break # Exit the attempt loop on ASCII error
        except HTTPError as e:
            print(f"⚠️ Retrying {player_slug} (Attempt {attempt}): URL error ({e.code})")
        except Exception as e:
            print(f"⚠️ Retrying {player_slug} (Attempt {attempt}): An unexpected error occurred: {e}")


    if not success and not is_ascii_error:
        failed_players.append(player_slug)
        print(f"❌ Failed: {player_slug} after {max_attempts} attempts.")
    elif is_ascii_error:
        failed_players.append(player_slug) # Add to failed_players if it was an ASCII error


print("\nFailed Players:", failed_players)

✅ Success: jacob-toppin (Attempt 1) - School matched: Kentucky
✅ Success: jalen-johnson (Attempt 1) - School matched: Duke
⚠️ Retrying keaton-wallace (Attempt 1): School not found in page.
⚠️ Retrying keaton-wallace (Attempt 2): URL error (404)
⚠️ Retrying keaton-wallace (Attempt 3): URL error (404)
⚠️ Retrying keaton-wallace (Attempt 4): URL error (404)
⚠️ Retrying keaton-wallace (Attempt 5): URL error (404)
❌ Failed: keaton-wallace after 5 attempts.
✅ Success: caris-levert (Attempt 1) - School matched: Michigan
✅ Success: kobe-bufkin (Attempt 1) - School matched: Michigan
⚠️ Retrying dyson-daniels (Attempt 1): URL error (404)
⚠️ Retrying dyson-daniels (Attempt 2): URL error (404)
⚠️ Retrying dyson-daniels (Attempt 3): URL error (404)
⚠️ Retrying dyson-daniels (Attempt 4): URL error (404)
⚠️ Retrying dyson-daniels (Attempt 5): URL error (404)
❌ Failed: dyson-daniels after 5 attempts.
⚠️ Retrying zaccharie-risacher (Attempt 1): URL error (404)
⚠️ Retrying zaccharie-risacher (Attempt 2)

In [ ]:
#based on what route, change the tag['id'] of the table
if route == 'college':
  player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='players_totals') #put the table in a variable
elif route == 'international':
  player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='player-stats-totals-all-')
  if player_table_soup == None:
    player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='player-stats-totals-league-')
    if player_table_soup == None:
      player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='player-stats-totals-tournament-')
    else:
      manual_input.append(next_player)
      #continue
elif route == 'gleague':
  player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='nbdl_totals-reg') #not accessing the table
  if player_table_soup == None:
    player_table_soup = soup.find('table',id='nbdl_totals-ex')

In [ ]:
table_headers = [th.getText() for th in player_table_soup.findAll('tr')[0].findAll('th')] #pull all the headers marked by th

rows_with_data = player_table_soup.findAll('tr')[1:] #pull which rows contain data

player_stats = [[td.getText() for td in rows_with_data [i].findAll('td')] #add each row to a dictionary to turn into a df
              for i in range(len(rows_with_data))]

for i in range(0, len(player_stats)):
            player_stats[i].insert(0, next_player)

table_headers.insert(0, "Name")

# create a dataframe with all aquired info
player_stats_final = pd.DataFrame(player_stats, columns = table_headers[1:])
player_stats_final.rename(columns = {'Season':'Name'}, inplace = True)

  # append new dataframe to final_df based on route.
if route == 'college':
  final_df_college = pd.concat([final_df_college, player_stats_final], ignore_index=True)
elif route == 'international':
    final_df_international = pd.concat([final_df_international, player_stats_final], ignore_index=True)
elif route == 'gleague':
    final_df_gleague = pd.concat([final_df_gleague, player_stats_final], ignore_index=True)

#outside of the big for loop, put manual_input into a df
manual_input_df = pd.DataFrame(manual_input, columns = ['Name'])


"final_df_college.to_csv('college_data.csv', index=False)\nfinal_df_international.to_csv('international_data.csv', index=False)\nfinal_df_gleague.to_csv('gleague_data.csv', index=False)\nmanual_input_df.to_csv('manual_input_df.csv', index=False)"

## Loop to scrape the data from the pages

The link of the player_slugs needs to be changed as well as the url to search for college players instead of international players.


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import pandas as pd
from google.colab import files, drive
from urllib.error import HTTPError, URLError
drive.mount('/content/drive')

final_df_college = pd.DataFrame()
player_slugs = pd.read_csv('https://docs.google.com/spreadsheets/d/1dTTbCpKBFposAELN6pr4bHgkaBCWKPLYTlqsMg5wAdw/gviz/tq?tqx=out:csv') #link changes based on college or international players
players = player_slugs[['PLAYER_SLUG']] #setting the parameter to run the scraping function

non_college_or_international = [] #failed players

for index, player in players.iterrows():
    player_slug = player['PLAYER_SLUG']
    time.sleep(6) #don't overload server

    url = f"https://www.basketball-reference.com/international/players/{player_slug}-1.html" #this link changes for international or college players
    try: html = urlopen(url)
    except(HTTPError, URLError):
        print(f"❌ Failed: {player_slug}")
        non_college_or_international.append(player_slug)
        continue
    soup = BeautifulSoup(html, 'lxml')

    player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='player-stats-totals-all-') #changed player_table_soup
    if player_table_soup == None:
        player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='player-stats-totals-league-')
        if player_table_soup == None:
          player_table_soup = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='player-stats-totals-tournament-')
        #check for None after trying all table IDs
        if player_table_soup is None:
            non_college_or_international.append(player_slug)
            print(f"❌ Failed to find table for: {player_slug}")
            continue

    table_headers = [th.getText() for th in player_table_soup.findAll('tr')[0].findAll('th')] #pull all the headers marked by th

    rows_with_data = player_table_soup.findAll('tr')[1:] #pull which rows contain data

    player_stats = [[td.getText() for td in rows_with_data [i].findAll('td')] #add each row to a dictionary to turn into a df
      for i in range(len(rows_with_data))]

    for i in range(0, len(player_stats)):
      player_stats[i].insert(0, player_slug) # Use player_slug instead of next_player

    table_headers.insert(0, "Name")

# create a dataframe with all aquired info
    player_stats_final = pd.DataFrame(player_stats, columns = table_headers[1:])
    player_stats_final.rename(columns = {'Season':'Name'}, inplace = True)

  # append new dataframe to final_df based on route.
    final_df_college = pd.concat([final_df_college, player_stats_final], ignore_index=True)
    print(f"✅ Success: {player_slug}")

final_df_college.to_csv('international_stats.csv', index=False)
non_college_or_international1 = pd.DataFrame(non_college_or_international, columns=['PLAYER_SLUG']) #add this csv
non_college_or_international1.to_csv("non_college_or_international.csv")
file_list = ['international_stats.csv', "non_college_or_international.csv"]


for file in file_list:
  !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
  print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-123599339.py:39: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table_headers = [th.getText() for th in player_table_soup.findAll('tr')[0].findAll('th')] #pull all the headers marked by th
/tmp/ipython-input-123599339.py:41: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  rows_with_data = player_table_soup.findAll('tr')[1:] #pull which rows contain data
/tmp/ipython-input-123599339.py:43: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  player_stats = [[td.getText() for td in rows_with_data [i].findAll('td')] #add each row to a dictionary to turn into a df


✅ Success: dyson-daniels
✅ Success: clint-capela
❌ Failed: dominick-barlow


#Clean sports-reference data


In [ ]:
import pandas as pd
import numpy as np
college_data = pd.read_csv('https://docs.google.com/spreadsheets/d/1KLlJ2mBWF2ynAT9tiWyN79JA9SVA0OhLSd5rt1wNLyg/gviz/tq?tqx=out:csv')

In [ ]:
college_data.rename(columns = {'School' : 'Team'}, inplace=True) #rename school to Team
college_data.loc[college_data['Team'] == 'Overall', 'Team'] = np.nan #make summary blank

In [ ]:
def find_valid_stopping_point(group):
    na_positions = group['Team'].isna()
    if not na_positions.any():
        return len(group)

    first_na_pos = na_positions.idxmax() #first team is NaN
    first_na_idx = group.index.get_loc(first_na_pos)

    row = group.loc[first_na_pos] #check if this row has NaNs in all columns except Name
    other_columns = [col for col in group.columns if col != 'Name']
    all_other_nan = row[other_columns].isna().all()

    if all_other_nan: #look for the next NaN team
        remaining_group = group.iloc[first_na_idx + 1:]
        next_na_positions = remaining_group['Team'].isna()

        if next_na_positions.any():
            next_na_pos = next_na_positions.idxmax()
            return group.index.get_loc(next_na_pos) + 1
        else:
            return len(group)
    else: #stop
        return first_na_idx + 1

college_overall_data = college_data.groupby('Name').apply(
    lambda x: x.head(find_valid_stopping_point(x))
).reset_index(drop=True)

/tmp/ipython-input-2889805318.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  college_overall_data = college_data.groupby('Name').apply(


In [ ]:
#list of columns to shift
cols_to_shift = ['Conf','Class','Pos','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for index, row in college_overall_data.iterrows():
    if pd.isna(row['Team']):
        print(f"Processing row {index} with NaN in 'Team'")

        shifted_values = row[cols_to_shift].shift(periods=3) #shift the data in the identified columns by 3 positions
        print(f"Shifted values: {shifted_values}")

        college_overall_data.loc[index, cols_to_shift] = shifted_values #update the row
        print(f"Row {index} after shifting: {college_overall_data.loc[index, cols_to_shift]}")

Processing row 4 with NaN in 'Team'
Shifted values: Conf       None
Class      None
Pos        None
G           115
GS          110
MP         3759
FG        500.0
FGA      1164.0
FG%        0.43
3P        205.0
3PA       548.0
3P%       0.374
2P        295.0
2PA       616.0
2P%       0.479
eFG%      0.518
FT        306.0
FTA       367.0
FT%       0.834
ORB        86.0
DRB       311.0
TRB       397.0
AST       470.0
STL       127.0
BLK        16.0
TOV       303.0
PF        265.0
PTS      1511.0
Name: 4, dtype: object
Row 4 after shifting: Conf       None
Class      None
Pos        None
G           115
GS          110
MP         3759
FG        500.0
FGA      1164.0
FG%        0.43
3P        205.0
3PA       548.0
3P%       0.374
2P        295.0
2PA       616.0
2P%       0.479
eFG%      0.518
FT        306.0
FTA       367.0
FT%       0.834
ORB        86.0
DRB       311.0
TRB       397.0
AST       470.0
STL       127.0
BLK        16.0
TOV       303.0
PF        265.0
PTS      1511.0
Name: 4

/tmp/ipython-input-376264983.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '115' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  college_overall_data.loc[index, cols_to_shift] = shifted_values
/tmp/ipython-input-376264983.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '110' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  college_overall_data.loc[index, cols_to_shift] = shifted_values
/tmp/ipython-input-376264983.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3759' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  college_overall_data.loc[index, cols_to_shift] = shifted_values


Streaming output truncated to the last 5000 lines.
2P%       0.523
eFG%      0.521
FT        346.0
FTA       444.0
FT%       0.779
ORB       161.0
DRB       339.0
TRB       685.0
AST       375.0
STL        90.0
BLK        55.0
TOV       283.0
PF        154.0
PTS      1626.0
Name: 4867, dtype: object
Row 4867 after shifting: Conf       None
Class      None
Pos        None
G            84
GS           57
MP         3044
FG        579.0
FGA      1228.0
FG%       0.471
3P        122.0
3PA       354.0
3P%       0.345
2P        457.0
2PA       874.0
2P%       0.523
eFG%      0.521
FT        346.0
FTA       444.0
FT%       0.779
ORB       161.0
DRB       339.0
TRB       685.0
AST       375.0
STL        90.0
BLK        55.0
TOV       283.0
PF        154.0
PTS      1626.0
Name: 4867, dtype: object
Processing row 4870 with NaN in 'Team'
Shifted values: Conf      None
Class     None
Pos       None
G           51
GS          25
MP        1097
FG       203.0
FGA      430.0
FG%      0.472
3P        

In [ ]:
def add_percentage_change_to_totals(dataframe):
    change_list = ['FG', '3P', '2P', 'FT']  # list to calculate change over time

    for name in dataframe['Name'].unique():
        temp_df = dataframe[dataframe['Name'] == name].copy()  # Use .copy() to avoid SettingWithCopyWarning

        # Find the index of the last row for the current player (which is the total row)
        total_row_index = temp_df.index[-1]

        for item in change_list:
            column_to_analyze = item + '%'  # check pct column
            filter_column = item + 'A'  # filter minimum attempts by using the 'A' column
            minimum_limit = 25  # a good cutoff point for minimum amount of attempts, I think

            # Fill NaN values with 0 before converting to integer and filtering
            temp_df[filter_column] = temp_df[filter_column].fillna(0)

            # Filter for rows with at least the minimum_limit attempts, excluding the total row
            filtered_df = temp_df.iloc[:-1][temp_df.iloc[:-1][filter_column].astype(int) >= minimum_limit]

            if len(filtered_df) >= 2:  # check if there are two rows to subtract
                difference = filtered_df[column_to_analyze].max() - filtered_df[column_to_analyze].min()  # subtract lowest value from highest value to (hopefully) determine growth
            else:
                difference = 0  # if not 2+ rows, set as 0

            # Add the calculated percentage change to the total row in the original dataframe
            dataframe.loc[total_row_index, item + '_pct_inc'] = round(difference, 3)

    return dataframe

In [ ]:
updated_college_data = add_percentage_change_to_totals(college_overall_data)
updated_college_data.head()

,Name,Team,Conf,Class,Pos,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,Awards,FG_pct_inc,3P_pct_inc,2P_pct_inc,FT_pct_inc
0,aaron-brooks-1,Oregon,Pac-10,FR,G,21.0,18.0,541.0,46.0,124.0,...,17.0,1.0,49.0,49.0,146.0,NaN,NaN,NaN,NaN,NaN
1,aaron-brooks-1,Oregon,Pac-10,SO,G,27.0,26.0,893.0,126.0,294.0,...,27.0,4.0,87.0,58.0,398.0,NaN,NaN,NaN,NaN,NaN
2,aaron-brooks-1,Oregon,Pac-10,JR,G,32.0,31.0,1036.0,115.0,283.0,...,35.0,5.0,78.0,70.0,346.0,NaN,NaN,NaN,NaN,NaN
3,aaron-brooks-1,Oregon,Pac-10,SR,G,35.0,35.0,1289.0,213.0,463.0,...,48.0,6.0,89.0,88.0,621.0,NaN,NaN,NaN,NaN,NaN
4,aaron-brooks-1,NaN,None,None,None,115,110,3759,500.0,1164.0,...,127.0,16.0,303.0,265.0,1511.0,NaN,0.089,0.068,0.127,0.067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173,zhaire-smith-1,NaN,None,None,None,37,21,1051,154.0,277.0,...,42.0,42.0,41.0,67.0,417.0,NaN,0.000,0.000,0.000,0.000
5174,ziaire-williams-1,Stanford,Pac-12,FR,F,20.0,14.0,558.0,76.0,203.0,...,17.0,11.0,58.0,48.0,214.0,NaN,NaN,NaN,NaN,NaN
5175,ziaire-williams-1,NaN,None,None,None,20,14,558,76.0,203.0,...,17.0,11.0,58.0,48.0,214.0,NaN,0.000,0.000,0.000,0.000
5176,zion-williamson-1,Duke,ACC,FR,F,33.0,33.0,990.0,296.0,435.0,...,70.0,59.0,78.0,68.0,746.0,"AP POY-1,WA-1,AA-1,AP-AA-1,NABC-AA-1,SN-AA-1,U...",NaN,NaN,NaN,NaN


In [ ]:
updated_college_data = updated_college_data.drop(['Team','Conf','Class','Pos','Awards'], axis=1) #drop the NaN columns
updated_college_data = updated_college_data.groupby('Name').tail(1).reset_index(drop=True)
updated_college_data = updated_college_data.reset_index(drop=True) #reset index
print(updated_college_data.shape) #check shape
updated_college_data.head()

(1317, 30)


,Name,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,AST,STL,BLK,TOV,PF,PTS,FG_pct_inc,3P_pct_inc,2P_pct_inc,FT_pct_inc
0,aaron-brooks-1,115,110,3759,500.0,1164.0,0.430,205.0,548.0,0.374,...,470.0,127.0,16.0,303.0,265.0,1511.0,0.089,0.068,0.127,0.067
1,aaron-gordon-1,38,38,1187,189.0,382.0,0.495,16.0,45.0,0.356,...,75.0,34.0,39.0,55.0,90.0,470.0,0.000,0.000,0.000,0.000
2,aaron-gray-1,113,68,2360,434.0,789.0,0.550,0.0,0.0,434.000,...,147.0,41.0,130.0,176.0,238.0,1109.0,0.050,0.000,0.050,0.086
3,aaron-harrison-1,79,78,2307,320.0,781.0,0.410,121.0,361.0,0.335,...,129.0,86.0,18.0,100.0,146.0,979.0,0.028,0.040,0.002,0.008
4,aaron-holiday-1,101,65,3209,476.0,1058.0,0.450,180.0,427.0,0.422,...,477.0,126.0,22.0,304.0,271.0,1443.0,0.091,0.018,0.156,0.101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,zach-randolph-1,33,8,654,138.0,235.0,0.587,0.0,1.0,0.000,...,34.0,24.0,22.0,53.0,60.0,356.0,0.000,0.000,0.000,0.000
1313,zeke-nnaji-1,32,32,981,179.0,314.0,0.570,5.0,17.0,0.294,...,27.0,23.0,28.0,70.0,76.0,515.0,0.000,0.000,0.000,0.000
1314,zhaire-smith-1,37,21,1051,154.0,277.0,0.556,18.0,40.0,0.450,...,66.0,42.0,42.0,41.0,67.0,417.0,0.000,0.000,0.000,0.000
1315,ziaire-williams-1,20,14,558,76.0,203.0,0.374,23.0,79.0,0.291,...,44.0,17.0,11.0,58.0,48.0,214.0,0.000,0.000,0.000,0.000


In [ ]:
import re

def clean_player_name(name): #change player names back to original format (since I forgot to bring them across in the previous steps)
    base_name = re.sub(r'-\d+$', '', name) #remove number
    player_slug = base_name #player_slug recreated
    cleaned_name = base_name.replace('-', ' ') #clean
    cleaned_name = cleaned_name.title()
    if cleaned_name.lower().endswith(' jr') or cleaned_name.lower().endswith('jr'): #check for jr. at end of name
      cleaned_name = re.sub(r'\s*Jr\.?$', ' Jr.', cleaned_name, flags=re.IGNORECASE).strip()
    return cleaned_name, player_slug

updated_college_data[['Name', 'Player_Slug']] = updated_college_data['Name'].apply(lambda x: pd.Series(clean_player_name(x))) #apply

print(updated_college_data[['Name', 'Player_Slug']].head())

             Name     Player_Slug
0    Aaron Brooks    aaron-brooks
1    Aaron Gordon    aaron-gordon
2      Aaron Gray      aaron-gray
3  Aaron Harrison  aaron-harrison
4   Aaron Holiday   aaron-holiday


In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')
updated_college_data.to_csv('updated_college_data.csv', index=False)
file_list = ['updated_college_data.csv']
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Mounted at /content/drive
Saved updated_college_data.csv to Drive


# RealGM Scraping Attempt

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import pandas as pd
from google.colab import files, drive
from urllib.error import HTTPError, URLError
drive.mount('/content/drive')

final_df_international = pd.DataFrame() #large df of around 3/4ths of all players
player_slugs = pd.read_csv('https://docs.google.com/spreadsheets/d/1dTTbCpKBFposAELN6pr4bHgkaBCWKPLYTlqsMg5wAdw/gviz/tq?tqx=out:csv') #changed link
players = player_slugs[['PLAYER_SLUG']] #setting the parameter to run the scraping function

non_college_or_international = [] #failed players

#add a user-gent header to mimic a browser request
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

last_successful_year = 2025


for index, player in players.iterrows():
    player_slug = player['PLAYER_SLUG']

    #change format for the page
    formatted_name = '-'.join(word.capitalize() for word in player_slug.split('-'))
    print(f"Processing player: {formatted_name}")

    time.sleep(6)
    get = False
    #search from the last successful year or 3 years prior
    start_year_search = min(2025, last_successful_year + 3)

    ending = None
    for year in range(start_year_search, 2004, -1): #search to 2005 matching NBA search
        url = f"https://basketball.realgm.com/nba/players/{year}"
        req = Request(url, headers=headers)
        try:
            html = urlopen(req)
            soup = BeautifulSoup(html, 'lxml')

            player_link_tag = soup.find('a', href=lambda href: href and f"/player/{formatted_name}/" in href)

            if player_link_tag:
                ending = player_link_tag.get('href')
                print(f"✅ Found player link for {formatted_name} in year {year}: {ending}")
                get = True
                last_successful_year = year #update last successful year
                break
            else:
                print(f"❌ Player link not found for {formatted_name} in year {year}. Trying previous year.")


        except HTTPError as e:
            print(f"❌ HTTP Error {e.code} for year page {url}. Trying previous year.")
        except URLError as e:
            print(f"❌ URL Error {e.reason} for year page {url}. Trying previous year.")
        except Exception as e:
            print(f"❌ An unexpected error occurred while searching for player link for {formatted_name} in year {year}: {e}")


    if not get:
        print(f"❌ Could not find player link for {formatted_name} in any year from {start_year_search} down to 2005.")
        non_college_or_international.append(player_slug)
        continue #next player

    player_url = f"https://basketball.realgm.com{ending}" #rebuild url
    print(f"✅ Accessing player page: {player_url}")

    try:
        time.sleep(6)
        req2 = Request(player_url, headers=headers)
        html2 = urlopen(req2)
        soup2 = BeautifulSoup(html2, 'lxml')


        player_table_soup = soup2.find(lambda tag: tag.name=='div' and tag.has_attr('id') and tag['id']=='tabs_international_reg-3') #changed player_table_soup

        if player_table_soup is None:
                non_college_or_international.append(player_slug)
                print(f"❌ Failed to find table for: {formatted_name}")
                continue
        else: print(f"✅ International Data found for: {formatted_name}")

        table_headers = [th.getText() for th in player_table_soup.find_all('tr')[0].find_all('th')]
        rows_with_data = player_table_soup.find_all('tr', class_='per_game')

        player_stats = []
        for row in rows_with_data:
            row_data = [td.getText() for td in row.find_all('td')]
            if len(row_data) == len(table_headers):
                player_stats.append(row_data)
            elif len(row_data) > 0:
                print(f"Skipping row with unexpected column count for {formatted_name}. Expected {len(table_headers)}, got {len(row_data)}. Data: {row_data}")


        print(f"Filtered rows: {len(player_stats)}")
        if len(player_stats) > 0:
            player_stats_final = pd.DataFrame(player_stats, columns = table_headers) #create new df
            player_stats_final.insert(0, 'Player_Slug', player_slug) #add name

            final_df_international = pd.concat([final_df_international, player_stats_final], ignore_index=True)
            print(f"✅ Successfully scraped data for: {formatted_name}")
        else:
            print(f"❌ No valid data rows found for {formatted_name} after filtering.")
            non_college_or_international.append(player_slug)

    except(HTTPError, URLError) as e:
        print(f"❌ Failed to access player page {player_url} for {formatted_name}: {e}")
        non_college_or_international.append(player_slug)
        continue #next player


final_df_international.to_csv('international_stats.csv', index=False)
non_college_or_international1 = pd.DataFrame(non_college_or_international, columns=['PLAYER_SLUG']) #add this csv
non_college_or_international1.to_csv("non_college_or_international.csv")
file_list = ['international_stats.csv', "non_college_or_international.csv"]


for file in file_list:
  !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
  print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing player: Dyson-Daniels
✅ Found player link for Dyson-Daniels in year 2025: /player/Dyson-Daniels/Summary/151792
✅ Accessing player page: https://basketball.realgm.com/player/Dyson-Daniels/Summary/151792
❌ Failed to find table for: Dyson-Daniels
Processing player: Clint-Capela
✅ Found player link for Clint-Capela in year 2025: /player/Clint-Capela/Summary/28284
✅ Accessing player page: https://basketball.realgm.com/player/Clint-Capela/Summary/28284
✅ International Data found for: Clint-Capela
Filtered rows: 7
✅ Successfully scraped data for: Clint-Capela
Processing player: Dominick-Barlow
✅ Found player link for Dominick-Barlow in year 2025: /player/Dominick-Barlow/Summary/180398
✅ Accessing player page: https://basketball.realgm.com/player/Dominick-Barlow/Summary/180398
❌ Failed to find table for: Dominick-Barlow
Processing player: Vit-Krejci
✅ Foun

# RealGM Cleaning

In [ ]:
import pandas as pd
import numpy as np

international_stats = pd.read_csv("https://docs.google.com/spreadsheets/d/1JJmzHpcQf36xxYGrl1rs2G-aYHnZyun2Czxrk-LeNOE/gviz/tq?tqx=out:csv")
new_column_names = ['Player_Slug','Season','Age','Team','League','GP','GS','MIN','PTS','FGM','FGA','FG%','3PM','3PA','3P%','FTM','FTA','FT%','OFF','DEF','TRB','AST','STL','BLK','TOV','PF']
international_stats.columns = new_column_names

In [ ]:
international_stats_2 = pd.DataFrame()
for index, row in international_stats.iterrows():
  if '*' not in str(row['Season']) or row['Team'] == 'All Teams':
    temp_row = pd.DataFrame([row], columns = international_stats.columns) #temp dataframe
    international_stats_2 = pd.concat([international_stats_2, temp_row], ignore_index=True)

international_stats_2

,Player_Slug,Season,Age,Team,League,GP,GS,MIN,PTS,FGM,...,FTA,FT%,OFF,DEF,TRB,AST,STL,BLK,TOV,PF
0,clint-capela,2011-12,17,Chalon-Sur-Saone U18,Kaunas,4,4,103.3,40,17,...,18,0.333,11,23,34,6,13,3,13,11
1,clint-capela,2012-13 *,18,All Teams,All Leagues,20,1,160.8,51,21,...,18,0.500,30,27,57,3,7,7,6,21
2,clint-capela,2013-14 *,19,All Teams,All Leagues,43,27,922.9,422,183,...,100,0.560,100,193,293,44,31,61,65,99
3,vit-krejci,2016-17,16,Casademont Zaragoza,ACB,1,0,0.2,0,0,...,0,0.000,0,0,0,0,0,0,0,0
4,vit-krejci,2018-19,18,Casademont Zaragoza,ACB,2,0,2.8,0,0,...,0,0.000,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,nikoloz-tskitishvili,2012-13,29,Champville,LEB DivA,28,28,"1,023.40",501,182,...,107,0.785,104,197,301,56,24,42,55,42
1285,nikoloz-tskitishvili,2014-15,31,Champville,LEB DivA,28,28,"1,047.80",556,173,...,188,0.878,101,212,313,59,14,30,44,58
1286,nikoloz-tskitishvili,2015-16 *,32,All Teams,All Leagues,25,19,706.3,374,132,...,109,0.807,71,139,210,35,29,12,39,56
1287,nikoloz-tskitishvili,2017-18 *,34,All Teams,All Leagues,10,9,332.1,188,65,...,63,0.730,37,40,77,7,11,11,24,17


In [ ]:
international_stats_2['MIN'] = international_stats_2['MIN'].astype(str).str.replace(',', '', regex=False) #convert to numeric
international_stats_2['MIN'] = pd.to_numeric(international_stats_2['MIN'], errors='coerce')

international_stats_3 = international_stats_2[international_stats_2['MIN'] > 100] #filter out years with less than 100 minutes
international_stats_3

,Player_Slug,Season,Age,Team,League,GP,GS,MIN,PTS,FGM,...,FTA,FT%,OFF,DEF,TRB,AST,STL,BLK,TOV,PF
0,clint-capela,2011-12,17,Chalon-Sur-Saone U18,Kaunas,4,4,103.3,40,17,...,18,0.333,11,23,34,6,13,3,13,11
1,clint-capela,2012-13 *,18,All Teams,All Leagues,20,1,160.8,51,21,...,18,0.500,30,27,57,3,7,7,6,21
2,clint-capela,2013-14 *,19,All Teams,All Leagues,43,27,922.9,422,183,...,100,0.560,100,193,293,44,31,61,65,99
5,vit-krejci,2019-20 *,19,All Teams,All Leagues,37,4,339.3,117,46,...,24,0.542,12,34,46,25,10,4,21,57
7,kristaps-porzingis,2011-12,16,Cajasol Seville U18,Ciutat De L'Hospitalet,5,4,136.4,46,18,...,7,0.714,10,14,24,5,5,10,11,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,nikoloz-tskitishvili,2012-13,29,Champville,LEB DivA,28,28,1023.4,501,182,...,107,0.785,104,197,301,56,24,42,55,42
1285,nikoloz-tskitishvili,2014-15,31,Champville,LEB DivA,28,28,1047.8,556,173,...,188,0.878,101,212,313,59,14,30,44,58
1286,nikoloz-tskitishvili,2015-16 *,32,All Teams,All Leagues,25,19,706.3,374,132,...,109,0.807,71,139,210,35,29,12,39,56
1287,nikoloz-tskitishvili,2017-18 *,34,All Teams,All Leagues,10,9,332.1,188,65,...,63,0.730,37,40,77,7,11,11,24,17


In [ ]:
cleaned_international_stats = pd.DataFrame()
players_removed_by_age = []


international_stats_3_sorted = international_stats_3.sort_values(by=['Player_Slug', 'Age']).copy() #looking to check for age gaps

for player_slug, group in international_stats_3_sorted.groupby('Player_Slug'):
    first_age = group['Age'].iloc[0]

    if first_age > 25:
        players_removed_by_age.append(player_slug)
        print(f"Removed player {player_slug} due to first age being {first_age}.")
        continue #skip to next player if age > 25 years

    ages = group['Age'].tolist()
    keep_rows = []
    last_age = None
    for i, age in enumerate(ages):
        if last_age is not None and age - last_age > 3: #remove age gaps greater than 3 years
            print(f"Removed rows after age {last_age} for player {player_slug} due to age gap.")
            break
        keep_rows.append(i)
        last_age = age

    cleaned_international_stats = pd.concat([cleaned_international_stats, group.iloc[keep_rows]])

print("\nPlayers removed due to initial age > 25:")
print(players_removed_by_age)

cleaned_international_stats.head()

Removed player al-harrington due to first age being 34.
Removed player al-jefferson due to first age being 33.
Removed rows after age 17 for player aleksej-pokusevski due to age gap.
Removed rows after age 22 for player alex-abrines due to age gap.
Removed rows after age 19 for player alexis-ajinca due to age gap.
Removed player amare-stoudemire due to first age being 33.
Removed rows after age 21 for player anderson-varejao due to age gap.
Removed player andray-blatche due to first age being 28.
Removed rows after age 20 for player andrea-bargnani due to age gap.
Removed player andrei-kirilenko due to first age being 30.
Removed rows after age 23 for player andres-nocioni due to age gap.
Removed rows after age 19 for player ante-zizic due to age gap.
Removed rows after age 21 for player beno-udrih due to age gap.
Removed rows after age 26 for player boban-marjanovic due to age gap.
Removed rows after age 20 for player boris-diaw due to age gap.
Removed rows after age 21 for player bos

,Player_Slug,Season,Age,Team,League,GP,GS,MIN,PTS,FGM,...,FTA,FT%,OFF,DEF,TRB,AST,STL,BLK,TOV,PF
318,adam-mokoka,2016-17 *,18,All Teams,All Leagues,20,1,160.4,43,15,...,10,0.800,1,8,9,4,4,0,9,11
319,adam-mokoka,2017-18,19,BCM Gravelines,Jeep Elite,34,21,476.0,109,39,...,38,0.553,26,33,59,31,14,3,39,55
320,adam-mokoka,2018-19 *,20,All Teams,All Leagues,32,32,908.0,355,123,...,104,0.644,42,79,121,119,41,8,90,107
321,adam-mokoka,2021-22,23,Nanterre 92,Jeep Elite,25,23,695.0,283,94,...,83,0.783,27,78,105,44,21,6,48,70
322,adam-mokoka,2022-23,24,Umana Venezia,Lega A,8,1,124.0,41,16,...,15,0.333,2,15,17,6,3,0,9,18


In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

players_removed_by_age = pd.DataFrame(players_removed_by_age, columns=['PLAYER_SLUG']) #add this csv
players_removed_by_age.to_csv("players_removed_by_age.csv")
file_list = ["players_removed_by_age.csv"]


for file in file_list:
  !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
  print(f"Saved {file} to Drive")

Mounted at /content/drive
Saved players_removed_by_age.csv to Drive


In [ ]:
def add_percentage_change_and_totals(dataframe):
    change_list = ['FG', '3P', 'FT']  # list to calculate change over time
    percentage_cols = ['FG%', '3P%', 'FT%'] # List of percentage columns

    # Create an empty DataFrame to store the results with total rows
    dataframe_with_totals = pd.DataFrame()

    for name, group in dataframe.groupby('Player_Slug'):
        temp_df = group.copy()  # Use .copy() to avoid SettingWithCopyWarning

        # Create a dictionary for the total row
        total_row_data = {'Player_Slug': name}

        # Initialize 'Season' through 'League' as empty
        for col in ['Season','Age','Team','League']:
            total_row_data[col] = None # Use None for missing values

        # Sum other numeric columns, handling non-numeric types by coercing to numeric first
        for col in temp_df.columns:
            if col not in ['Player_Slug','Season','Age','Team','League'] + percentage_cols:
                # Coerce to numeric, errors='coerce' will turn non-numeric into NaN
                numeric_col = pd.to_numeric(temp_df[col], errors='coerce')
                total_row_data[col] = numeric_col.sum()

        # Calculate percentages for the total row
        for item in change_list:
            makes_col = item + 'M'
            attempts_col = item + 'A'
            total_makes = pd.to_numeric(temp_df[makes_col], errors='coerce').sum()
            total_attempts = pd.to_numeric(temp_df[attempts_col], errors='coerce').sum()
            total_row_data[item + '%'] = round(total_makes / total_attempts, 3) if total_attempts > 0 else 0 # Calculate percentage and round

        # Create a DataFrame for the total row
        total_df = pd.DataFrame([total_row_data])

        # Concatenate the original player data with the total row
        temp_df = pd.concat([temp_df, total_df], ignore_index=True)

        # Calculate percentage increase for the total row (from the original function logic)
        total_row_index = temp_df.index[-1]
        for item in change_list:
            column_to_analyze = item + '%'  # check pct column
            filter_column = item + 'A'  # filter minimum attempts by using the 'A' column
            minimum_limit = 25  # a good cutoff point for minimum amount of attempts, I think

            # Fill NaN values with 0 before converting to integer and filtering for percentage increase calculation
            temp_df[filter_column] = pd.to_numeric(temp_df[filter_column], errors='coerce').fillna(0)

            # Filter for rows with at least the minimum_limit attempts, excluding the total row
            filtered_df = temp_df.iloc[:-1][temp_df.iloc[:-1][filter_column].astype(int) >= minimum_limit]

            if len(filtered_df) >= 2:  # check if there are two rows to subtract
                # Calculate the difference using the max and min of the percentage column in filtered rows
                difference = filtered_df[column_to_analyze].max() - filtered_df[column_to_analyze].min()
            else:
                difference = 0  # if not 2+ rows, set as 0

            # Add the calculated percentage change to the total row
            temp_df.loc[total_row_index, item + '_pct_inc'] = round(difference, 3)


        # Append the processed player data (with total row and pct_inc) to the main results DataFrame
        dataframe_with_totals = pd.concat([dataframe_with_totals, temp_df], ignore_index=True)

    return dataframe_with_totals

In [ ]:
international_stats_4 = add_percentage_change_and_totals(cleaned_international_stats) #run it

In [ ]:
international_totals_only = international_stats_4[international_stats_4['Season'].isna()].reset_index(drop=True) #reset index
international_totals_only.head()

,Player_Slug,Season,Age,Team,League,GP,GS,MIN,PTS,FGM,...,DEF,TRB,AST,STL,BLK,TOV,PF,FG_pct_inc,3P_pct_inc,FT_pct_inc
0,adam-mokoka,None,None,None,None,226,118,4743.9,1920,661,...,458,628,471,162,60,359,500,0.139,0.115,0.230
1,aleksej-pokusevski,None,None,None,None,18,12,367.1,183,64,...,99,117,45,24,34,32,26,0.023,0.000,0.000
2,alen-smailagic,None,None,None,None,248,116,4040.1,1828,664,...,450,748,188,163,129,221,585,0.064,0.148,0.095
3,alex-abrines,None,None,None,None,288,81,4908.7,2066,698,...,348,482,226,179,59,185,514,0.057,0.114,0.117
4,alexey-shved,None,None,None,None,507,401,13618.7,6373.0,2817,...,1381,1607,2886,522,164,1359,876,0.085,0.108,0.133


In [ ]:
international_totals = international_totals_only.assign(**{'2P':np.nan,'2PA':np.nan,'2P%':np.nan,'eFG%':np.nan,'2P_pct_inc':np.nan}) #fill the missing columns with NaNs
international_totals.head()


,Player_Slug,Season,Age,Team,League,GP,GS,MIN,PTS,FGM,...,TOV,PF,FG_pct_inc,3P_pct_inc,FT_pct_inc,2P,2PA,2P%,eFG%,2P_pct_inc
0,adam-mokoka,None,None,None,None,226,118,4743.9,1920,661,...,359,500,0.139,0.115,0.230,NaN,NaN,NaN,NaN,NaN
1,aleksej-pokusevski,None,None,None,None,18,12,367.1,183,64,...,32,26,0.023,0.000,0.000,NaN,NaN,NaN,NaN,NaN
2,alen-smailagic,None,None,None,None,248,116,4040.1,1828,664,...,221,585,0.064,0.148,0.095,NaN,NaN,NaN,NaN,NaN
3,alex-abrines,None,None,None,None,288,81,4908.7,2066,698,...,185,514,0.057,0.114,0.117,NaN,NaN,NaN,NaN,NaN
4,alexey-shved,None,None,None,None,507,401,13618.7,6373.0,2817,...,1359,876,0.085,0.108,0.133,NaN,NaN,NaN,NaN,NaN


In [ ]:
desired_order = ['Name','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','FG_pct_inc','3P_pct_inc','2P_pct_inc','FT_pct_inc']

#rename columns
international_totals_renamed = international_totals.rename(columns={'Player_Slug':'Name','GP': 'G', 'MIN':'MP','FGM':'FG','3PM':'3P','FTM':'FT','OFF':'ORB','DEF':'DRB'})

all_columns = set(international_totals_renamed.columns)
desired_columns = set(desired_order)
columns_to_drop = list(all_columns - desired_columns)

international_totals_cleaned = international_totals_renamed.drop(columns=columns_to_drop) #drop the unneeded columns
international_totals = international_totals_cleaned[desired_order]

print("Columns dropped:")
for col in columns_to_drop: #just to double check
    print(col)


international_totals.head()

Columns dropped:
Season
League
Age
Team


,Name,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,AST,STL,BLK,TOV,PF,PTS,FG_pct_inc,3P_pct_inc,2P_pct_inc,FT_pct_inc
0,adam-mokoka,226,118,4743.9,661,1511,0.437,198,646,0.307,...,471,162,60,359,500,1920,0.139,0.115,NaN,0.230
1,aleksej-pokusevski,18,12,367.1,64,157,0.408,27,73,0.370,...,45,24,34,32,26,183,0.023,0.000,NaN,0.000
2,alen-smailagic,248,116,4040.1,664,1184,0.561,165,433,0.381,...,188,163,129,221,585,1828,0.064,0.148,NaN,0.095
3,alex-abrines,288,81,4908.7,698,1549,0.451,367,947,0.388,...,226,179,59,185,514,2066,0.057,0.114,NaN,0.117
4,alexey-shved,507,401,13618.7,2817,6721,0.419,1258,3646,0.345,...,2886,522,164,1359,876,6373.0,0.085,0.108,NaN,0.133


In [1]:
import re

def clean_player_name_int(name): #unfortunately have to fix all of the player names since I forgot to carry them over
    player_slug = name

    cleaned_name = name.replace('-', ' ')
    cleaned_name = cleaned_name.title()
    if cleaned_name.lower().endswith(' jr') or cleaned_name.lower().endswith('jr'):
      cleaned_name = re.sub(r'\s*Jr\.?$', ' Jr.', cleaned_name, flags=re.IGNORECASE).strip()
    return cleaned_name, player_slug

international_totals[['Name', 'Player_Slug']] = international_totals['Name'].apply(lambda x: pd.Series(clean_player_name_int(x)))

print(international_totals[['Name', 'Player_Slug']].head())

NameError: name 'international_totals' is not defined

In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

international_totals.to_csv("international_totals_cleaned.csv")

file_list = ['international_totals_cleaned.csv']
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved international_totals_cleaned.csv to Drive


In [ ]:
combined_data = pd.concat([updated_college_data, international_totals], ignore_index=True) #stack vertically


display(combined_data.head())
display(combined_data.info()) #check datatypes

,Name,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,STL,BLK,TOV,PF,PTS,FG_pct_inc,3P_pct_inc,2P_pct_inc,FT_pct_inc,Player_Slug
0,Aaron Brooks,115,110,3759,500.0,1164.0,0.430,205.0,548.0,0.374,...,127.0,16.0,303.0,265.0,1511.0,0.089,0.068,0.127,0.067,aaron-brooks
1,Aaron Gordon,38,38,1187,189.0,382.0,0.495,16.0,45.0,0.356,...,34.0,39.0,55.0,90.0,470.0,0.000,0.000,0.000,0.000,aaron-gordon
2,Aaron Gray,113,68,2360,434.0,789.0,0.550,0.0,0.0,434.000,...,41.0,130.0,176.0,238.0,1109.0,0.050,0.000,0.050,0.086,aaron-gray
3,Aaron Harrison,79,78,2307,320.0,781.0,0.410,121.0,361.0,0.335,...,86.0,18.0,100.0,146.0,979.0,0.028,0.040,0.002,0.008,aaron-harrison
4,Aaron Holiday,101,65,3209,476.0,1058.0,0.450,180.0,427.0,0.422,...,126.0,22.0,304.0,271.0,1443.0,0.091,0.018,0.156,0.101,aaron-holiday


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         1482 non-null   object 
 1   G            1482 non-null   object 
 2   GS           1482 non-null   object 
 3   MP           1482 non-null   object 
 4   FG           1482 non-null   float64
 5   FGA          1475 non-null   float64
 6   FG%          1482 non-null   float64
 7   3P           1482 non-null   float64
 8   3PA          1482 non-null   float64
 9   3P%          1482 non-null   float64
 10  2P           1315 non-null   float64
 11  2PA          1315 non-null   float64
 12  2P%          1245 non-null   float64
 13  eFG%         1317 non-null   float64
 14  FT           1482 non-null   float64
 15  FTA          1480 non-null   float64
 16  FT%          1480 non-null   float64
 17  ORB          1482 non-null   float64
 18  DRB          1482 non-null   float64
 19  TRB   

None

In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

combined_data.to_csv("combined_data_int_col_data.csv")

file_list = ['combined_data_int_col_data.csv']
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved combined_data_int_col_data.csv to Drive


# Dataset Linking

When linking the college data to the NBA data, there is going to be issues linking on the names of players with . in their first name or hyphens in last name.

In [ ]:
import pandas as pd
import numpy as np

player_info = pd.read_csv('https://docs.google.com/spreadsheets/d/1EREZxYx81CN-PLCJ71ErG1x4lb4XnVtbseuMz0BSwao/gviz/tq?tqx=out:csv')
nba_data = pd.read_csv('https://docs.google.com/spreadsheets/d/1KZUS5oSUvaUHmQZfMZ20RT3bOgOo6WQDBge6Mf8R0hI/gviz/tq?tqx=out:csv')
pre_nba_data = pd.read_csv('https://docs.google.com/spreadsheets/d/1X7y6ZnZ9dq4ChypfN4xFuxcNBhPjSZDk6mi4ZeMGdEk/gviz/tq?tqx=out:csv')

In [ ]:
#merge the two dataframes on PLAYER_ID
nba_data.columns = ['NBA_' + col for col in nba_data.columns]
merged_data = pd.merge(player_info, nba_data, left_on='PLAYER_ID', right_on="NBA_PLAYER_ID", how='left')
merged_data.head()

,Unnamed: 0,SEASON,PLAYER,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,...,NBA_AST,NBA_STL,NBA_BLK,NBA_TOV,NBA_PF,NBA_PTS,NBA_FG_pct_inc,NBA_FG3_pct_inc,NBA_FT_pct_inc,NBA_Team_ID
0,0,2024,Jacob Toppin,jacob-toppin,0.0,F,6-8,200,"May 08, 2000",25,...,9,2,1,4,6,36,0.000,0.000,0.000,NaN
1,1,2024,Jalen Johnson,jalen-johnson,1.0,F,6-8,219,"Dec 18, 2001",23,...,470,164,119,257,322,2023,-0.033,0.038,0.084,NaN
2,2,2024,Caris LeVert,caris-levert,3.0,G,6-6,205,"Aug 25, 1994",30,...,2105,548,198,918,1034,7274,-0.010,0.023,0.023,NaN
3,3,2024,Kobe Bufkin,kobe-bufkin,4.0,G,6-4,195,"Sep 21, 2003",21,...,44,10,7,18,29,134,0.004,0.000,0.000,NaN
4,4,2024,Dyson Daniels,dyson-daniels,5.0,G,6-7,199,"Mar 17, 2003",22,...,631,357,91,271,375,1654,0.054,0.013,-0.036,NaN


In [ ]:
#check for dupes
duplicates = pre_nba_data[pre_nba_data.duplicated(subset=['PLAYER_SLUG'], keep=False)]

if not duplicates.empty:
    print("Duplicate rows found in 'PRE_PLAYER_SLUG' column:")
    display(duplicates)
else:
    print("No duplicate rows found in 'PRE_PLAYER_SLUG' column.")

#I fixed all of these instances manually because it was far more simple than writing a code to do so

No duplicate rows found in 'PRE_PLAYER_SLUG' column.


In [ ]:
#add 'PRE_' prefix to all column names in pre_nba_data
pre_nba_data.columns = ['PRE_' + col for col in pre_nba_data.columns]
merged_data_full = pd.merge(merged_data, pre_nba_data, left_on='PLAYER_SLUG', right_on='PRE_PLAYER_SLUG', how='left') #merge
merged_data_full.head()

,Unnamed: 0,SEASON,PLAYER,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,...,PRE_BLK,PRE_TOV,PRE_PF,PRE_PTS,PRE_FG_pct_inc,PRE_3P_pct_inc,PRE_2P_pct_inc,PRE_FT_pct_inc,PRE_PLAYER_SLUG,PRE_COL_or_INT
0,0,2024,Jacob Toppin,jacob-toppin,0.0,F,6-8,200,"May 08, 2000",25,...,50.0,103.0,199.0,864.0,0.130,0.060,0.104,0.136,jacob-toppin,COL
1,1,2024,Jalen Johnson,jalen-johnson,1.0,F,6-8,219,"Dec 18, 2001",23,...,16.0,33.0,29.0,146.0,0.000,0.000,0.000,0.000,jalen-johnson,COL
2,2,2024,Caris LeVert,caris-levert,3.0,G,6-6,205,"Aug 25, 1994",30,...,22.0,136.0,151.0,1070.0,0.191,0.144,0.220,0.043,caris-levert,COL
3,3,2024,Kobe Bufkin,kobe-bufkin,4.0,G,6-4,195,"Sep 21, 2003",21,...,26.0,79.0,104.0,547.0,0.102,0.133,0.034,0.000,kobe-bufkin,COL
4,4,2024,Dyson Daniels,dyson-daniels,5.0,G,6-7,199,"Mar 17, 2003",22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

merged_data_full.to_csv("full_df_1.csv")

file_list = ['full_df_1.csv']
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved full_df_1.csv to Drive


# Dataset Cleaning

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("https://docs.google.com/spreadsheets/d/1pdL2qyPUNWP2NNNzJCMgKbPrsVOBFQ7eMGSpfJfBbbs/gviz/tq?tqx=out:csv")

In [ ]:
#drop duplicate rows based on 'PLAYER_ID' and 'SEASON', keeping the first occurrence
data_cleaned = data.drop_duplicates(subset=['PLAYER_ID', 'SEASON'], keep='first')
print(f"Number of rows after dropping duplicates: {len(data_cleaned)}")
display(data_cleaned.head())

Number of rows after dropping duplicates: 1573


,PLAYER_ID,SEASON,PLAYER,PLAYER_SLUG,NUM,POSITION,WEIGHT,BIRTH_DATE,AGE,EXP,...,PRE_STL,PRE_BLK,PRE_TOV,PRE_PF,PRE_PTS,PRE_FG_pct_inc,PRE_3P_pct_inc,PRE_2P_pct_inc,PRE_FT_pct_inc,PRE_COL_or_INT
0,1631210,2024,Jacob Toppin,jacob-toppin,0.0,F,200,"May 08, 2000",25,1,...,48.0,50.0,103.0,199.0,864.0,0.130,0.060,0.104,0.136,COL
1,1630552,2024,Jalen Johnson,jalen-johnson,1.0,F,219,"Dec 18, 2001",23,3,...,15.0,16.0,33.0,29.0,146.0,0.000,0.000,0.000,0.000,COL
2,1627747,2024,Caris LeVert,caris-levert,3.0,G,205,"Aug 25, 1994",30,8,...,97.0,22.0,136.0,151.0,1070.0,0.191,0.144,0.220,0.043,COL
3,1641723,2024,Kobe Bufkin,kobe-bufkin,4.0,G,195,"Sep 21, 2003",21,1,...,54.0,26.0,79.0,104.0,547.0,0.102,0.133,0.034,0.000,COL
4,1630700,2024,Dyson Daniels,dyson-daniels,5.0,G,199,"Mar 17, 2003",22,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#filter out rows where 'PRE_COL_or_INT' is NaN
data_with_pre_data = data_cleaned.dropna(subset=['PRE_COL_or_INT']).reset_index(drop=True)

#list of dropped players
dropped_players_no_pre_data = data_cleaned[data_cleaned['PRE_COL_or_INT'].isna()]['PLAYER'].unique().tolist()

print(f"Number of rows after dropping players with no pre-NBA data: {len(data_with_pre_data)}")
print(f"Number of players dropped due to no pre-NBA data: {len(dropped_players_no_pre_data)}")
print("\nPlayers dropped due to no pre-NBA data:")
print(dropped_players_no_pre_data)

display(data_with_pre_data.head())

Number of rows after dropping players with no pre-NBA data: 1479
Number of players dropped due to no pre-NBA data: 94

Players dropped due to no pre-NBA data:
['Dyson Daniels', 'Dominick Barlow', 'Danté Exum', 'Jaden Hardy', 'Jonathan Kuminga', 'Amen Thompson', 'Jalen Green', 'Patrick Baldwin Jr.', 'LeBron James', 'Giannis Antetokounmpo', 'Leonard Miller', 'MarJon Beauchamp', 'Mitchell Robinson', 'Mac McClung', 'Scoot Henderson', 'Anfernee Simons', 'Shaedon Sharpe', 'KJ Martin', 'Ausar Thompson', 'LaMelo Ball', 'Daishen Nix', 'Raul Neto', 'Isaiah Todd', 'R.J. Hampton', 'Lou Williams', 'Brodric Thomas', 'Jay Scrubb', 'Dwight Howard', 'Enes Freedom', 'Tyson Chandler', 'JR Smith', 'Dirk Nowitzki', 'Shaun Livingston', 'Nene', 'Amir Johnson', 'CJ Miles', 'Tony Parker', 'Kendrick Perkins', 'Al Jefferson', 'Georgios Papagiannis', 'Monta Ellis', 'Leandro Barbosa', 'Josh Smith', 'Pablo Prigioni', 'Jeff Ayres', 'Kobe Bryant', "Amar'e Stoudemire", 'Dorell Wright', 'Kevin Garnett', 'Chris Andersen

,PLAYER_ID,SEASON,PLAYER,PLAYER_SLUG,NUM,POSITION,WEIGHT,BIRTH_DATE,AGE,EXP,...,PRE_STL,PRE_BLK,PRE_TOV,PRE_PF,PRE_PTS,PRE_FG_pct_inc,PRE_3P_pct_inc,PRE_2P_pct_inc,PRE_FT_pct_inc,PRE_COL_or_INT
0,1631210,2024,Jacob Toppin,jacob-toppin,0.0,F,200,"May 08, 2000",25,1,...,48.0,50.0,103.0,199.0,864.0,0.130,0.060,0.104,0.136,COL
1,1630552,2024,Jalen Johnson,jalen-johnson,1.0,F,219,"Dec 18, 2001",23,3,...,15.0,16.0,33.0,29.0,146.0,0.000,0.000,0.000,0.000,COL
2,1627747,2024,Caris LeVert,caris-levert,3.0,G,205,"Aug 25, 1994",30,8,...,97.0,22.0,136.0,151.0,1070.0,0.191,0.144,0.220,0.043,COL
3,1641723,2024,Kobe Bufkin,kobe-bufkin,4.0,G,195,"Sep 21, 2003",21,1,...,54.0,26.0,79.0,104.0,547.0,0.102,0.133,0.034,0.000,COL
4,1629027,2024,Trae Young,trae-young,11.0,G,164,"Sep 19, 1998",26,6,...,54.0,8.0,167.0,57.0,876.0,0.000,0.000,0.000,0.000,COL


In [ ]:
dropped_players_no_pre_data = pd.DataFrame(dropped_players_no_pre_data, columns=['Name']) #add this csv
dropped_players_no_pre_data.to_csv("dropped_players_no_pre_data.csv")


from google.colab import files, drive
drive.mount('/content/drive')

data_with_pre_data.to_csv("data_with_pre_data.csv")

file_list = ['data_with_pre_data.csv']
for file in file_list:
      !cp "{file}" "/content/drive/My Drive/Junior Year/Thesis/"
      print(f"Saved {file} to Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved data_with_pre_data.csv to Drive
